# **Finger 2**

In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
import os
from heapq import nlargest

os.environ['PYSPARK_PYTHON'] = '/usr/bin/python3'

In [2]:
conf = SparkConf().setAppName('Finger 2').setMaster('local')
sc = SparkContext(conf=conf)
sqlContext = SQLContext(sc)

In [3]:
df = sqlContext.read.csv('events.csv')

# **Punto 1**: tipo de envento predominante

In [4]:
evento, frecuencia = df.rdd.map(lambda x: (x[1], 1)).reduceByKey(lambda x, y: x + y).reduce(lambda x, y: x if (x[1] > y[1]) else y)
print("El evento predominante es '{}', con {} apariciones.".format(evento, frecuencia))

El evento predominante es 'viewed product', con 528931 apariciones.


# **Punto 2**: cinco smartphones mas visitados

In [5]:
df.rdd.filter(lambda x: (x[1] == 'viewed product')) \
            .map(lambda x: (x[5], 1)).reduceByKey(lambda x, y: x + y).takeOrdered(5, key=lambda x: -x[1])

[('iPhone 6', 47538),
 ('iPhone 5s', 42045),
 ('iPhone 6S', 36930),
 ('iPhone 7', 26330),
 ('Samsung Galaxy S7 Edge', 23062)]

Este metodo no es muy eficiente, ya que ordena todo. El siguente metodo carga las cantidades de los distintos modelos en memoria y utiliza nlargest para obtener los 5 mayores (O(n log(5)).

In [6]:
visitas = df.rdd.filter(lambda x: (x[1] == 'viewed product')) \
            .map(lambda x: (x[5], 1)).reduceByKey(lambda x, y: x + y).collect()

In [7]:
nlargest(5, visitas, key=lambda x: x[1])

[('iPhone 6', 47538),
 ('iPhone 5s', 42045),
 ('iPhone 6S', 36930),
 ('iPhone 7', 26330),
 ('Samsung Galaxy S7 Edge', 23062)]

### Comparacion de los metodos

In [8]:
%%timeit
%%capture
df.rdd.filter(lambda x: (x[1] == 'viewed product')) \
            .map(lambda x: (x[5], 1)).reduceByKey(lambda x, y: x + y).takeOrdered(5, key=lambda x: -x[1])

9.21 s ± 117 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
%%timeit
df.rdd.filter(lambda x: (x[1] == 'viewed product')) \
            .map(lambda x: (x[5], 1)).reduceByKey(lambda x, y: x + y).collect()
nlargest(5, visitas, key=lambda x: x[1])

9.17 s ± 55.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


El segundo metodo es mas eficiente en cuanto a velocidad, aunque por poco, seguramente debido a que la lista de resultados no es muy larga.
Sin embargo, si uno no puede cargar la lista de modelos por cantidad en memoria, se veria obligado a utilizar el primer metodo.